# Cognitive Workspace Monorepo Structure Verification

This notebook contains tests to verify the correct setup of the monorepo structure for the Cognitive Workspace application.

In [ ]:
%pip install termcolor

import os
import json
import subprocess
import sys
from pathlib import Path
import unittest
# from termcolor import colored

## Directory Structure Tests

These tests verify the existence of core directories in the monorepo structure.

In [ ]:
class DirectoryStructureTests(unittest.TestCase):
    def setUp(self):
        # Navigate to project root (assuming this notebook is in the project root)
        self.project_root = Path(os.getcwd())

    def test_core_directories_exist(self):
        """Test that all core directories exist"""
        required_dirs = ['backend', 'frontend', 'shared', 'infrastructure']

        for dir_name in required_dirs:
            dir_path = self.project_root / dir_name
            self.assertTrue(dir_path.exists(), f"Directory '{dir_name}' does not exist")
            self.assertTrue(dir_path.is_dir(), f"'{dir_name}' is not a directory")

    def test_backend_subdirectories(self):
        """Test that backend has appropriate subdirectories"""
        backend_dir = self.project_root / 'backend'
        # Assuming backend follows a modular structure
        expected_subdirs = ['api', 'core', 'infrastructure', 'scripts', 'tests']

        for subdir in expected_subdirs:
            subdir_path = backend_dir / subdir
            self.assertTrue(subdir_path.exists(), f"Backend subdirectory '{subdir}' does not exist")
            self.assertTrue(subdir_path.is_dir(), f"Backend '{subdir}' is not a directory")

    def test_frontend_subdirectories(self):
        """Test that frontend has appropriate subdirectories"""
        frontend_dir = self.project_root / 'frontend'
        expected_subdirs = ['src', 'public', 'tests']

        for subdir in expected_subdirs:
            subdir_path = frontend_dir / subdir
            self.assertTrue(subdir_path.exists(), f"Frontend subdirectory '{subdir}' does not exist")
            self.assertTrue(subdir_path.is_dir(), f"Frontend '{subdir}' is not a directory")

    def test_shared_directory_structure(self):
        """Test that shared directory has appropriate structure"""
        shared_dir = self.project_root / 'shared'
        expected_subdirs = ['types', 'utils', 'constants']

        for subdir in expected_subdirs:
            subdir_path = shared_dir / subdir
            self.assertTrue(subdir_path.exists(), f"Shared subdirectory '{subdir}' does not exist")
            self.assertTrue(subdir_path.is_dir(), f"Shared '{subdir}' is not a directory")

    def test_infrastructure_directory_structure(self):
        """Test that infrastructure directory has appropriate structure"""
        infra_dir = self.project_root / 'infrastructure'
        expected_subdirs = ['terraform', 'docker', 'scripts']

        for subdir in expected_subdirs:
            subdir_path = infra_dir / subdir
            self.assertTrue(subdir_path.exists(), f"Infrastructure subdirectory '{subdir}' does not exist")
            self.assertTrue(subdir_path.is_dir(), f"Infrastructure '{subdir}' is not a directory")

## Configuration Files Tests

These tests verify the existence and correctness of essential configuration files.

In [ ]:
class ConfigurationFilesTests(unittest.TestCase):
    def setUp(self):
        self.project_root = Path(os.getcwd())

    def test_root_configuration_files_exist(self):
        """Test that essential root configuration files exist"""
        required_files = [
            'package.json',
            'tsconfig.json',
            '.gitignore',
            'README.md',
            '.prettierrc',
            '.eslintrc.js',
            '.husky/pre-commit'
        ]

        for file_name in required_files:
            file_path = self.project_root / file_name
            self.assertTrue(file_path.exists(), f"Configuration file '{file_name}' does not exist")

    def test_package_json_workspaces(self):
        """Test that package.json has workspaces configuration"""
        with open(self.project_root / 'package.json', 'r') as f:
            package_json = json.load(f)

        self.assertIn('workspaces', package_json, "package.json does not have workspaces configuration")
        workspaces = package_json['workspaces']
        expected_workspaces = ['frontend', 'shared', 'infrastructure']

        for workspace in expected_workspaces:
            self.assertTrue(any(ws.startswith(workspace) for ws in workspaces),
                           f"Workspace '{workspace}' not configured in package.json")

    def test_typescript_configuration(self):
        """Test that TypeScript configuration is properly set up"""
        with open(self.project_root / 'tsconfig.json', 'r') as f:
            tsconfig = json.load(f)

        self.assertIn('compilerOptions', tsconfig, "tsconfig.json does not have compilerOptions")
        self.assertIn('references', tsconfig, "tsconfig.json does not have project references")

        # Check for strict mode
        compiler_options = tsconfig['compilerOptions']
        self.assertTrue(compiler_options.get('strict', False), "TypeScript strict mode is not enabled")

    def test_gitignore_configuration(self):
        """Test that .gitignore has essential entries"""
        with open(self.project_root / '.gitignore', 'r') as f:
            gitignore_content = f.read()

        essential_ignores = ['node_modules', 'dist', 'build', '.env', 'coverage', '.DS_Store']
        for entry in essential_ignores:
            self.assertIn(entry, gitignore_content, f"'{entry}' not found in .gitignore")

    def test_readme_content(self):
        """Test that README.md has essential sections"""
        with open(self.project_root / 'README.md', 'r') as f:
            readme_content = f.read()

        essential_sections = ['# Cognitive Workspace', 'Installation', 'Usage', 'Development', 'Structure']
        for section in essential_sections:
            self.assertIn(section, readme_content, f"'{section}' section not found in README.md")

    def test_eslint_configuration(self):
        """Test that ESLint is properly configured"""
        eslint_path = self.project_root / '.eslintrc.js'
        self.assertTrue(eslint_path.exists(), "ESLint configuration file does not exist")

        # Read as text since it's a JS file, not JSON
        with open(eslint_path, 'r') as f:
            eslint_content = f.read()

        essential_configs = ['typescript', 'prettier', 'react']
        for config in essential_configs:
            self.assertIn(config, eslint_content.lower(), f"ESLint configuration for '{config}' not found")

    def test_prettier_configuration(self):
        """Test that Prettier is properly configured"""
        prettier_path = self.project_root / '.prettierrc'
        self.assertTrue(prettier_path.exists(), "Prettier configuration file does not exist")

        with open(prettier_path, 'r') as f:
            prettier_config = json.load(f)

        essential_keys = ['tabWidth', 'semi', 'singleQuote', 'printWidth']
        for key in essential_keys:
            self.assertIn(key, prettier_config, f"Prettier configuration key '{key}' not found")

    def test_husky_configuration(self):
        """Test that Husky pre-commit hook is configured"""
        husky_path = self.project_root / '.husky' / 'pre-commit'
        self.assertTrue(husky_path.exists(), "Husky pre-commit hook does not exist")

        with open(husky_path, 'r') as f:
            hook_content = f.read()

        # Check for common pre-commit tasks
        common_tasks = ['lint-staged', 'prettier', 'eslint', 'test']
        found_tasks = 0
        for task in common_tasks:
            if task in hook_content.lower():
                found_tasks += 1

        self.assertGreater(found_tasks, 0, "No common pre-commit tasks found in Husky configuration")

    def test_backend_python_configuration(self):
        """Test that Python backend has proper configuration"""
        backend_configs = [
            'backend/pyproject.toml',
            'backend/.flake8',
            'backend/setup.py'
        ]

        for config in backend_configs:
            config_path = self.project_root / config
            self.assertTrue(config_path.exists(), f"Backend configuration '{config}' does not exist")

## Build Process Tests

These tests verify that the monorepo can be built without errors.

In [ ]:
class BuildProcessTests(unittest.TestCase):
    def setUp(self):
        self.project_root = Path(os.getcwd())

    def test_npm_install(self):
        """Test that npm install runs without errors"""
        try:
            # Run npm install with --dry-run to check for errors without actually installing
            result = subprocess.run(['npm', 'install', '--dry-run'],
                                   cwd=self.project_root,
                                   check=True,
                                   capture_output=True,
                                   text=True)
            self.assertEqual(result.returncode, 0, "npm install --dry-run failed")
        except subprocess.CalledProcessError as e:
            self.fail(f"npm install --dry-run failed with error: {e.stderr}")

    def test_typescript_compilation(self):
        """Test that TypeScript code compiles without errors"""
        try:
            # Run TypeScript compilation check without emitting files
            result = subprocess.run(['npx', 'tsc', '--noEmit'],
                                   cwd=self.project_root,
                                   check=True,
                                   capture_output=True,
                                   text=True)
            self.assertEqual(result.returncode, 0, "TypeScript compilation failed")
        except subprocess.CalledProcessError as e:
            self.fail(f"TypeScript compilation failed with error: {e.stderr}")

    def test_build_script(self):
        """Test that build script exists and runs"""
        with open(self.project_root / 'package.json', 'r') as f:
            package_json = json.load(f)

        self.assertIn('scripts', package_json, "package.json does not have scripts section")
        self.assertIn('build', package_json['scripts'], "package.json does not have build script")

        # Don't actually run the build as it might be time-consuming
        # Just check that the script is defined
        build_script = package_json['scripts']['build']
        self.assertTrue(build_script, "Build script is empty")

    def test_backend_build(self):
        """Test that Python backend can be built/installed"""
        backend_dir = self.project_root / 'backend'
        if not (backend_dir / 'pyproject.toml').exists():
            self.skipTest("Backend pyproject.toml not found, skipping backend build test")

        try:
            # Check that pip install would work, without actually installing
            result = subprocess.run(['pip', 'install', '-e', '.', '--dry-run'],
                                   cwd=backend_dir,
                                   check=True,
                                   capture_output=True,
                                   text=True)
            self.assertEqual(result.returncode, 0, "Backend pip install failed")
        except subprocess.CalledProcessError as e:
            self.fail(f"Backend pip install failed with error: {e.stderr}")

## Linting and Type Checking Tests

These tests verify that linting and type checking are properly configured.

In [ ]:
class LintingAndTypeCheckingTests(unittest.TestCase):
    def setUp(self):
        self.project_root = Path(os.getcwd())

    def test_eslint_runs(self):
        """Test that ESLint runs without fatal errors"""
        try:
            # Check if ESLint is configured properly by running it in dry mode
            result = subprocess.run(['npx', 'eslint', '--no-eslintrc', '--print-config', '.'],
                                   cwd=self.project_root,
                                   check=True,
                                   capture_output=True,
                                   text=True)
            self.assertEqual(result.returncode, 0, "ESLint configuration check failed")
        except subprocess.CalledProcessError as e:
            self.fail(f"ESLint configuration is invalid: {e.stderr}")

    def test_prettier_runs(self):
        """Test that Prettier runs without fatal errors"""
        try:
            # Check if Prettier is configured properly by running it in check mode
            result = subprocess.run(['npx', 'prettier', '--check', '--loglevel', 'error', '.prettierrc'],
                                   cwd=self.project_root,
                                   check=False,  # Don't fail if files aren't formatted
                                   capture_output=True,
                                   text=True)
            self.assertEqual(result.returncode, 0, "Prettier configuration check failed")
        except subprocess.CalledProcessError as e:
            self.fail(f"Prettier configuration is invalid: {e.stderr}")

    def test_typescript_project_references(self):
        """Test that TypeScript project references are properly configured"""
        with open(self.project_root / 'tsconfig.json', 'r') as f:
            tsconfig = json.load(f)

        self.assertIn('references', tsconfig, "tsconfig.json does not have project references")
        references = tsconfig['references']
        self.assertTrue(isinstance(references, list), "references in tsconfig.json is not an array")
        self.assertTrue(len(references) > 0, "No project references found in tsconfig.json")

        # Check that the referenced projects exist
        for ref in references:
            if 'path' in ref:
                ref_path = self.project_root / ref['path'] / 'tsconfig.json'
                self.assertTrue(ref_path.exists(), f"Referenced project at '{ref['path']}' does not exist")

    def test_flake8_configuration(self):
        """Test that Flake8 is properly configured for Python code"""
        flake8_path = self.project_root / 'backend' / '.flake8'
        if not flake8_path.exists():
            self.skipTest("Flake8 configuration file not found, skipping test")

        with open(flake8_path, 'r') as f:
            flake8_content = f.read()

        # Check for basic flake8 configuration sections
        self.assertIn('[flake8]', flake8_content, "Flake8 configuration is missing [flake8] section")

    def test_python_typing(self):
        """Test that Python code has type hints configuration"""
        pyproject_path = self.project_root / 'backend' / 'pyproject.toml'
        if not pyproject_path.exists():
            self.skipTest("pyproject.toml not found, skipping Python typing test")

        with open(pyproject_path, 'r') as f:
            pyproject_content = f.read()

        # Check for mypy or some kind of typing tool configuration
        typing_tools = ['mypy', 'typing', 'pyre', 'pyright']
        typing_configured = any(tool in pyproject_content.lower() for tool in typing_tools)
        self.assertTrue(typing_configured, "No Python typing tool configuration found in pyproject.toml")

## Run All Tests

In [ ]:
def run_tests():
    """Run all test classes and report results"""
    print(colored("\n=== Cognitive Workspace Monorepo Structure Verification ===\n", "cyan"))

    test_classes = [
        DirectoryStructureTests,
        ConfigurationFilesTests,
        BuildProcessTests,
        LintingAndTypeCheckingTests
    ]

    overall_success = True

    for test_class in test_classes:
        suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
        result = unittest.TextTestRunner(verbosity=2).run(suite)

        if not result.wasSuccessful():
            overall_success = False

        print("\n")

    if overall_success:
        print(colored("✅ All monorepo structure tests passed! The monorepo is correctly set up.", "green"))
    else:
        print(colored("❌ Some monorepo structure tests failed. Please fix the issues before proceeding.", "red"))

    return overall_success

In [ ]:
if __name__ == "__main__":
    success = run_tests()
    if not success:
        sys.exit(1)

## Recommendations and Next Steps

Based on the test results, here are recommended actions to ensure your monorepo is fully set up:

1. For any failing directory structure tests:
   - Create the missing core directories
   - Ensure the basic structure within each directory is correct

2. For any failing configuration file tests:
   - Add missing configuration files
   - Correct any malformed JSON in existing files
   - Ensure all required fields are present in package.json files

3. For build process failures:
   - Check package.json scripts for correct build commands
   - Resolve any dependency issues
   - Fix build errors in the codebase

4. For linting issues:
   - Add missing ESLint or Prettier configurations
   - Resolve any lint errors in the codebase

5. For type checking issues:
   - Add missing TypeScript configurations
   - Fix any type errors in the codebase

Run this notebook again after making changes to verify that all issues have been resolved.